# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
# Your code here; create the connection
conn = sqlite3.Connection("data/data2.sqlite")

### Q.1 Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [38]:
# Your code here
q1 = """
SELECT 
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
WHERE customerNumber IN
    (SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31')
;
"""
pd.read_sql(q1, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


### Q.2 Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [22]:
# Your code here
q2 = """
SELECT 
    p.productName, 
    SUM(od.quantityOrdered) AS total_number_orders
FROM orderdetails od
JOIN products p 
    ON od.productCode = p.productCode
GROUP BY p.productName
ORDER BY total_number_orders DESC
;
"""
pd.read_sql(q2,conn)

,productName,total_number_orders
0,1992 Ferrari 360 Spider red,1808
1,1937 Lincoln Berline,1111
2,American Airlines: MD-11S,1085
3,1941 Chevrolet Special Deluxe Cabriolet,1076
4,1930 Buick Marquette Phaeton,1074
...,...,...
104,1999 Indy 500 Monte Carlo SS,855
105,1911 Ford Town Car,832
106,1936 Mercedes Benz 500k Roadster,824
107,1970 Chevy Chevelle SS 454,803


### Q.3 Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

**A quick note on the SQL  `SELECT DISTINCT` statement:**

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [28]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q3 = """
SELECT DISTINCT
    p.productName, 
    COUNT( o.customerNumber) AS total_customers
FROM orderdetails od
JOIN orders o 
    ON od.orderNumber = o.orderNumber
JOIN products p 
    ON od.productCode = p.productCode
GROUP BY p.productName
ORDER BY total_customers DESC
;
"""
pd.read_sql(q3,conn)

,productName,total_customers
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
...,...,...
104,1932 Alfa Romeo 8C2300 Spider Sport,25
105,1917 Grand Touring Sedan,25
106,1911 Ford Town Car,25
107,1957 Ford Thunderbird,24


### Q.4 Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [36]:
# Your code here
q4 = """
SELECT DISTINCT 
    e.employeeNumber, 
    e.firstName, 
    e.lastName, 
    o.city, 
    e.officeCode
FROM employees e
JOIN customers c 
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o 
    ON c.customerNumber = o.customerNumber
JOIN orderdetails od 
    ON o.orderNumber = od.orderNumber
JOIN products p 
    ON od.productCode = p.productCode
WHERE p.productCode IN (
    SELECT p.productCode
    FROM orderdetails od
    JOIN orders o ON od.orderNumber = o.orderNumber
    GROUP BY p.productCode
    HAVING COUNT(o.customerNumber) < 20)
;
"""
pd.read_sql(q4,conn)

DatabaseError: Execution failed on sql '
SELECT DISTINCT 
    e.employeeNumber, 
    e.firstName, 
    e.lastName, 
    o.city, 
    e.officeCode
FROM employees e
JOIN customers c 
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o 
    ON c.customerNumber = o.customerNumber
JOIN orderdetails od 
    ON o.orderNumber = od.orderNumber
JOIN products p 
    ON od.productCode = p.productCode
WHERE p.productCode IN (
    SELECT p.productCode
    FROM orderdetails od
    JOIN orders o ON od.orderNumber = o.orderNumber
    GROUP BY p.productCode
    HAVING COUNT(o.customerNumber) < 20)
;
': no such column: o.city

### Q.5 Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [46]:
# Your code here
q4 = """
SELECT 
    e.employeeNumber, 
    e.firstName, 
    e.lastName, 
    COUNT(c.customerNumber) AS total_customers,
    c.creditLimit
FROM employees e
JOIN customers c 
    ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING AVG(c.creditLimit) > 15000
;
"""
pd.read_sql(q4,conn)

,employeeNumber,firstName,lastName,total_customers,creditLimit
0,1165,Leslie,Jennings,6,210500
1,1166,Leslie,Thompson,6,71800
2,1188,Julie,Firrelli,6,43400
3,1216,Steve,Patterson,6,100600
4,1286,Foon Yue,Tseng,7,138500
5,1323,George,Vanauf,8,114900
6,1337,Loui,Bondur,6,123900
7,1370,Gerard,Hernandez,7,21000
8,1401,Pamela,Castillo,10,83400
9,1501,Larry,Bott,8,96500


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!